# Basic Examples of Metrics with Prometheus Operator

## Prerequisites

 * A kubernetes cluster with kubectl configured
 * curl
 

## Setup Seldon Core

Install Seldon Core as described in [docs](https://docs.seldon.io/projects/seldon-core/en/latest/workflow/install.html).

Then port-forward to that ingress on localhost:8003 in a separate terminal either with:
```bash
kubectl port-forward -n istio-system svc/istio-ingressgateway 8080:8080
```

In [179]:
!kubectl create namespace seldon || echo "Seldon namespace already exists"

Error from server (AlreadyExists): namespaces "seldon" already exists
Seldon namespace already exists
Seldon namespace already exists


## Install Prometheus Operator

In [180]:
!kubectl create namespace seldon-monitoring
!helm upgrade --install seldon-monitoring kube-prometheus-stack \
    --version 44.4.1 \
    --set fullnameOverride=seldon-monitoring \
    --set prometheus.scrapeInterval=1s \
    --namespace seldon-monitoring \
    --repo https://prometheus-community.github.io/helm-charts/ \
    --wait

namespace/seldon-monitoring created
Release "seldon-monitoring" does not exist. Installing it now.
Release "seldon-monitoring" does not exist. Installing it now.
NAME: seldon-monitoring
LAST DEPLOYED: Wed Dec 10 15:07:37 2025
NAMESPACE: seldon-monitoring
STATUS: deployed
REVISION: 1
NOTES:
kube-prometheus-stack has been installed. Check its status by running:
  kubectl --namespace seldon-monitoring get pods -l "release=seldon-monitoring"

Visit https://github.com/prometheus-operator/kube-prometheus for instructions on how to create & configure Alertmanager and Prometheus instances using the Operator.
NAME: seldon-monitoring
LAST DEPLOYED: Wed Dec 10 15:07:37 2025
NAMESPACE: seldon-monitoring
STATUS: deployed
REVISION: 1
NOTES:
kube-prometheus-stack has been installed. Check its status by running:
  kubectl --namespace seldon-monitoring get pods -l "release=seldon-monitoring"

Visit https://github.com/prometheus-operator/kube-prometheus for instructions on how to create & configure Aler

In [181]:
%%bash
kubectl rollout status -n seldon-monitoring deployment/seldon-monitoring-operator
kubectl rollout status -n seldon-monitoring statefulsets/prometheus-seldon-monitoring-prometheus

deployment "seldon-monitoring-operator" successfully rolled out
statefulset rolling update complete 1 pods at revision prometheus-seldon-monitoring-prometheus-58fb79649...
statefulset rolling update complete 1 pods at revision prometheus-seldon-monitoring-prometheus-58fb79649...


In [182]:
%%bash
cat <<EOF | kubectl apply -f -
apiVersion: monitoring.coreos.com/v1
kind: PodMonitor
metadata:
  name: seldon-podmonitor
  namespace: seldon-monitoring
  labels:
    release: seldon-monitoring
spec:
  selector:
    matchLabels:
      seldon-app: echo-default
  podMetricsEndpoints:
    - port: metrics
      path: /prometheus
  namespaceSelector:
    matchNames:
      - seldon
EOF


podmonitor.monitoring.coreos.com/seldon-podmonitor created


## Deploy Example Model

In [183]:
%%writefile echo-sdep.yaml
apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: echo
  namespace: seldon
spec:
  predictors:
  - name: default
    replicas: 1
    graph:
      name: classifier
      type: MODEL
    componentSpecs:
    - spec:
        containers:
        - image: seldonio/echo-model:1.19.0-rc.1
          name: classifier


Overwriting echo-sdep.yaml


In [184]:
!kubectl apply -f echo-sdep.yaml

seldondeployment.machinelearning.seldon.io/echo created


In [185]:
!kubectl wait sdep/echo \
  --for=condition=ready \
  --timeout=120s \
  -n seldon

seldondeployment.machinelearning.seldon.io/echo condition met


## Sent series of REST requests

In [186]:
%%bash

# Wait for the model to become fully ready
echo "Waiting 5s for model to fully ready"
sleep 5

# Send 20 requests to REST endpoint
for i in `seq 1 10`; do sleep 0.1 && \
   curl -s -H "Content-Type: application/json" \
   -d '{"data": {"ndarray":[[1.0, 2.0, 5.0]]}}' \
   http://localhost:8080/seldon/seldon/echo/api/v1.0/predictions > /dev/null ; \
done

# Give time for metrics to get collected by Prometheus
echo "Waiting 10s for Prometheus to scrape metrics"
sleep 10

Waiting 5s for model to fully ready
Waiting 10s for Prometheus to scrape metrics
Waiting 10s for Prometheus to scrape metrics


## Check Metrics (REST)

In [187]:
import json

In [188]:
%%writefile get-metrics.sh
QUERY='query=seldon_api_executor_client_requests_seconds_count{model_name="classifier",namespace="seldon"}'
QUERY_URL=http://seldon-monitoring-prometheus.seldon-monitoring.svc.cluster.local:9090/api/v1/query

kubectl run -n seldon --quiet=true -it --rm curlmetrics-$(date +%s) --image=curlimages/curl:8.6.0 --restart=Never -- \
    curl --data-urlencode ${QUERY} ${QUERY_URL}


Overwriting get-metrics.sh


In [190]:
metrics = ! bash get-metrics.sh
metrics

['{"status":"success","data":{"resultType":"vector","result":[{"metric":{"__name__":"seldon_api_executor_client_requests_seconds_count","code":"200","container":"seldon-container-engine","deployment_name":"echo","endpoint":"metrics","instance":"10.244.1.244:8000","job":"seldon-monitoring/seldon-podmonitor","method":"post","model_image":"seldonio/echo-model","model_name":"classifier","model_version":"1.19.0-dev","namespace":"seldon","pod":"echo-default-0-classifier-69b9bd5787-7d5m8","predictor_name":"default","service":"/predict"},"value":[1765379371.016,"10"]}]}}']

In [191]:
metrics = json.loads(metrics[0])

In [192]:
counter = int(metrics["data"]["result"][0]["value"][1])
assert counter == 10, f"expected 10 requests, got {counter}"

## Send series GRPC requests

In [194]:
%%bash
cd ../../../executor/proto && for i in `seq 1 10`; do sleep 0.1 && \
    grpcurl -d '{"data": {"ndarray":[[1.0, 2.0, 5.0]]}}' \
    -rpc-header seldon:echo -rpc-header namespace:seldon \
    -plaintext -proto ./prediction.proto \
     0.0.0.0:8080 seldon.protos.Seldon/Predict > /dev/null ; \
done

# Give time for metrics to get collected by Prometheus
echo "Waiting 10s for Prometheus to scrape metrics"
sleep 10

Waiting 10s for Prometheus to scrape metrics


## Check metrics (GRPC)

In [200]:
%%writefile get-metrics.sh
QUERY='query=seldon_api_executor_client_requests_seconds_count{model_name="classifier",namespace="seldon",method=~"unary"}'
QUERY_URL=http://seldon-monitoring-prometheus.seldon-monitoring.svc.cluster.local:9090/api/v1/query

kubectl run -n seldon --quiet=true -it --rm curlmetrics-$(date +%s) --image=curlimages/curl:8.6.0 --restart=Never -- \
    curl --data-urlencode ${QUERY} ${QUERY_URL}


Overwriting get-metrics.sh


In [201]:
metrics = ! bash get-metrics.sh
metrics

['{"status":"success","data":{"resultType":"vector","result":[{"metric":{"__name__":"seldon_api_executor_client_requests_seconds_count","code":"OK","container":"seldon-container-engine","deployment_name":"echo","endpoint":"metrics","instance":"10.244.1.244:8000","job":"seldon-monitoring/seldon-podmonitor","method":"unary","model_image":"seldonio/echo-model","model_name":"classifier","model_version":"1.19.0-dev","namespace":"seldon","pod":"echo-default-0-classifier-69b9bd5787-7d5m8","predictor_name":"default","service":"/seldon.protos.Model/Predict"},"value":[1765379489.472,"10"]}]}}']

In [202]:
metrics = json.loads(metrics[0])

In [203]:
counter = int(metrics["data"]["result"][0]["value"][1])
assert counter == 10, f"expected 10 requests, got {counter}"

## Check Custom Metrics

This model defines a few custom metrics in its `.py` class definition:
```Python
    def metrics(self):
        print("metrics called")
        return [
            # a counter which will increase by the given value
            {"type": "COUNTER", "key": "mycounter", "value": 1},

            # a gauge which will be set to given value
            {"type": "GAUGE", "key": "mygauge", "value": 100},

            # a timer (in msecs) which  will be aggregated into HISTOGRAM
            {"type": "TIMER", "key": "mytimer", "value": 20.2},
        ]
```      

We will be checking value of `mygaguge` metrics.

In [204]:
%%writefile get-metrics.sh
QUERY='query=mygauge{model_name="classifier",namespace="seldon"}'
QUERY_URL=http://seldon-monitoring-prometheus.seldon-monitoring.svc.cluster.local:9090/api/v1/query

kubectl run -n seldon --quiet=true -it --rm curlmetrics-$(date +%s) --image=curlimages/curl:8.6.0 --restart=Never -- \
    curl --data-urlencode ${QUERY} ${QUERY_URL}


Overwriting get-metrics.sh


In [205]:
metrics = ! bash get-metrics.sh
metrics

['{"status":"success","data":{"resultType":"vector","result":[{"metric":{"__name__":"mygauge","container":"classifier","deployment_name":"echo","endpoint":"metrics","image_name":"seldonio/echo-model","image_version":"1.19.0-dev","instance":"10.244.1.244:6000","job":"seldon-monitoring/seldon-podmonitor","method":"predict","model_image":"seldonio/echo-model","model_name":"classifier","model_version":"1.19.0-dev","namespace":"seldon","pod":"echo-default-0-classifier-69b9bd5787-7d5m8","predictor_name":"default","predictor_version":"default","seldon_deployment_name":"echo","worker_id":"112"},"value":[1765379541.097,"100"]},{"metric":{"__name__":"mygauge","container":"classifier","deployment_name":"echo","endpoint":"metrics","image_name":"seldonio/echo-model","image_version":"1.19.0-dev","instance":"10.244.1.244:6000","job":"seldon-monitoring/seldon-podmonitor","method":"predict","model_image":"seldonio/echo-model","model_name":"classifier","model_version":"1.19.0-dev","namespace":"seldon","

In [206]:
metrics = json.loads(metrics[0])

In [207]:
gauge = int(metrics["data"]["result"][0]["value"][1])
assert gauge == 100, f"expected 100 on guage, got {gauge}"

## Cleanup

In [208]:
!kubectl delete sdep -n seldon echo
!helm uninstall -n seldon-monitoring seldon-monitoring

seldondeployment.machinelearning.seldon.io "echo" deleted
release "seldon-monitoring" uninstalled
release "seldon-monitoring" uninstalled
